In [ ]:
! pip install collections-extended
! pip install langdetect
! pip install googletrans
! pip install contractions
! pip install unidecode
! pip install googletrans==3.1.0a0
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=65fb780399ba68e08b1a651a8b155d8987a84ed6094a8c61787cd18bf4fe695d
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.2 MB/s eta 0:00:00
     

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# importing libraries

import pandas as pd
from langdetect import detect
from googletrans import Translator
import contractions
from unidecode import unidecode
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [ ]:
  df1 = pd.read_csv('dataset_es_train.csv',on_bad_lines='skip',engine = 'python')
  df1.head(2)

review_id          product_id          reviewer_id  stars  \
0  es_0491108  product_es_0296024  reviewer_es_0999081      1   
1  es_0869872  product_es_0922286  reviewer_es_0216771      1   

                                         review_body  \
0  Nada bueno se me fue ka pantalla en menos de 8...   
1  Horrible, nos tuvimos que comprar otro porque ...   

                                review_title language product_category  
0                           television Nevir       es      electronics  
1  Dinero tirado a la basura con esta compra       es      electronics

In [ ]:
sample = df1.sample(1000)
sample.reset_index(drop=True, inplace=True)

In [ ]:
data = sample[['review_body','stars']]

In [ ]:
def lang_trans(data):
    translor = Translator()
    translated_text = translor.translate(data)
    return translated_text.text

In [ ]:
data['translated_reviews'] = data.review_body.apply(lang_trans)

<ipython-input-7-5f20d40599fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['translated_reviews'] = data.review_body.apply(lang_trans)


In [ ]:
data

review_body  stars  \
0    Libro muy interesante, aunque a veces, los cue...      4   
1    Bonito, original y perfecto. No muy durables p...      3   
2    Recomendable. Son de buena calidad. Vienen 3 m...      4   
3    Mi perro ha tenido una cirugía en su pierna, p...      2   
4    Funciona correctamente. El sistema de sujeción...      4   
..                                                 ...    ...   
995  La falda talla bastante pequeña, necesitaría u...      2   
996  Fatal, la tinta de color solo funciona dos col...      2   
997  Venia el paquete con muchos golpes, no del tra...      2   
998  Muy buena funda, se ajusta perfectamente al mó...      5   
999  La manguera me ha durado 1 mes y ya se rompió....      1   

                                    translated_reviews  
0    Very interesting book, although sometimes, the...  
1    Beautiful, original and perfect. Not very dura...  
2    Recommendable. They are good quality. They com...  
3    My dog ​​has had surgery on his leg, but this ...  
4    Works correctly. The fastening system does not...  
..                                                 ...  
995  The skirt is quite small, I would need a size ...  
996  Terrible, the color ink only works in two of t...  
997  The package came with many blows, not from tra...  
998  Very good case, both the glass and the casing ...  
999  The hose has lasted me 1 month and it has alre...  

[1000 rows x 3 columns]

In [ ]:
data= data[['translated_reviews','stars']]


In [ ]:
data.stars.unique()

array([4, 3, 2, 5, 1])

In [ ]:
data.stars.value_counts()

2    213
3    206
1    198
5    197
4    186
Name: stars, dtype: int64

In [ ]:
# remove newlines
def remove_lines(data):
    clean_text =  data.replace("\n",' ').replace("\\n",' ').replace("\t",' ')
    return clean_text

# contraction mapping
def expand_text(data):
    expanded_doc = contractions.fix(data)
    return expanded_doc

# handle accented character

def accented_char(data):
    fixed_text = unidecode(data)
    return fixed_text

# clean data \
stopword_list = stopwords.words("english")
stopword_list.remove("not")
stopword_list.remove("no")
stopword_list.remove("nor")
def clean_data(data):
    tokens = word_tokenize(data)
    normalization = [word.lower() for word in tokens]
    remove_punct = [word for word in normalization if word not in punctuation]
    words_without_stop = [word for word in  remove_punct if word not in stopword_list]
    clean_text = [word for word in words_without_stop if len(word)>2]
    return clean_text

# lemmatization
def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    final_text = []
    for word in data:
        lemmatized_word = lemmatizer.lemmatize(word)
        final_text.append(lemmatized_word)
    return final_text

def join_list(data):
    return " ".join(data)

In [ ]:
def target_column(data):
    if (data==1) or (data==2):
        return 0
    elif (data==3):
        return 1
    elif (data==4) or (data==5):
        return 2
data['target'] = data.stars.apply(target_column)

In [ ]:
data['target']

0      2
1      1
2      2
3      0
4      2
      ..
995    0
996    0
997    0
998    2
999    0
Name: target, Length: 1000, dtype: int64

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data.translated_reviews,data.target,test_size=0.25,random_state=42)
clean_text_train = x_train.apply(remove_lines)

clean_text_train = clean_text_train.apply(expand_text)

clean_text_train = clean_text_train.apply(accented_char)

clean_text_train = clean_text_train.apply(clean_data)

clean_text_train = clean_text_train.apply(lemmatization)

clean_text_train = clean_text_train.apply(join_list)


In [ ]:
count = CountVectorizer(max_df=0.95,max_features=1000)
count_train_val = count.fit_transform(clean_text_train)


In [ ]:
mnb_count = MultinomialNB()
mnb_count.fit(count_train_val.A,y_train)

MultinomialNB()

In [ ]:
import pickle
pickle.dump(count, open('count2.pkl', "wb"))

In [ ]:
import pickle
pickle.dump(mnb_count, open('model2.pkl', "wb"))